In [1]:
import sys
sys.path.append('..')

import os
import os.path as osp
import time
import pickle
import numpy as np
import pandas as pd
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TerminateOnNaN, TensorBoard
from sklearn.model_selection import train_test_split

# custom modules
from libs.utilities import load_dataset, create_autoencoder_model, load_autoencoder_model, load_autoencoder_lossfunc
from libs.model_utils import LossLayer
from libs.data_generator import DataGenerator
from libs.processing import pink_noise, s_to_power


Using TensorFlow backend.


In [2]:
#### PARAMETERS ####

model_source = './example_configfile2.json'


In [3]:
#### OTHER VARS ####
# loss function: data slice under consideration
time_slice = slice(None)
initial_epoch = 0
input_shape = (256, 16, 1)


In [4]:
### SHOW MODEL FILE ###
import json
import pprint 

pprint.pprint(json.load(open(model_source)))

{'architecture': [{'layer_args': {'filters': 64,
                                  'kernel_size': [256, 1],
                                  'padding': 'valid',
                                  'strides': 1},
                   'layer_type': 'conv'},
                  {'layer_args': {}, 'layer_type': 'batchnorm'},
                  {'layer_args': {'activation': 'relu'},
                   'layer_type': 'activation'},
                  {'layer_args': {'rate': 0.2}, 'layer_type': 'dropout'},
                  {'layer_args': {'filters': 128,
                                  'kernel_size': [1, 3],
                                  'padding': 'same',
                                  'strides': [1, 2]},
                   'layer_type': 'conv'},
                  {'layer_args': {}, 'layer_type': 'batchnorm'},
                  {'layer_args': {'activation': 'relu'},
                   'layer_type': 'activation'},
                  {'layer_args': {'rate': 0.2}, 'layer_type': 'dropout'},
   

In [5]:
### TRY AND GENERATE MODEL ###
# NOTE: restart the kernel and rerun everything after editing an imported script or lib
model, lossfunc = create_autoencoder_model(
    model_source, input_shape, time_slice=time_slice)

[u] Creating autoencoder model from ./example_configfile2.json
[u] Model factory parameters: {'model_architecture': {'architecture': [{'layer_type': 'conv', 'layer_args': {'filters': 64, 'kernel_size': [256, 1], 'padding': 'valid', 'strides': 1}}, {'layer_type': 'batchnorm', 'layer_args': {}}, {'layer_type': 'activation', 'layer_args': {'activation': 'relu'}}, {'layer_type': 'dropout', 'layer_args': {'rate': 0.2}}, {'layer_type': 'conv', 'layer_args': {'filters': 128, 'kernel_size': [1, 3], 'padding': 'same', 'strides': [1, 2]}}, {'layer_type': 'batchnorm', 'layer_args': {}}, {'layer_type': 'activation', 'layer_args': {'activation': 'relu'}}, {'layer_type': 'dropout', 'layer_args': {'rate': 0.2}}, {'layer_type': 'conv', 'layer_args': {'filters': 256, 'kernel_size': [1, 3], 'padding': 'same', 'strides': [1, 2]}}, {'layer_type': 'batchnorm', 'layer_args': {}}, {'layer_type': 'activation', 'layer_args': {'activation': 'relu'}}, {'layer_type': 'dropout', 'layer_args': {'rate': 0.2}}, {'lay

In [7]:
### VERIFY GENERATED MODEL ###
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 16, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 16, 64)         16448     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 16, 64)         256       
_________________________________________________________________
activation_1 (Activation)    (None, 1, 16, 64)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 16, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 8, 128)         24704     
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 8, 128)         512       
__________

_____
# Can we use the built-in Keras Model exporters (dict, yaml, json)?

In [18]:
### TEST KERAS BUILTIN JSON IMPORT/EXPORT
from models.model_example import AEModelFactory
obj = AEModelFactory(input_shape, 3, 16)
model2 = obj.get_model()

In [19]:
model2.get_layer('encoder').summary()
model2.get_layer('decoder').summary()
model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 256, 16, 1)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 8, 4)         40        
_________________________________________________________________
batch_normalization_23 (Batc (None, 128, 8, 4)         16        
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 4, 8)          296       
_________________________________________________________________
batch_normalization_24 (Batc (None, 64, 4, 8)          32        
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 2, 16)         1168      
_________________________________________________________________
batch_normalization_25 (Batc (None, 32, 2, 16)         64        
__________

In [17]:
from pprint import pprint
model_config = model2.get_config()
print(model_config)
#pprint(model_config)  ### looks prettier but takes all the space

{'name': 'model_6', 'layers': [{'name': 'input', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 256, 16, 1), 'dtype': 'float32', 'sparse': False, 'name': 'input'}, 'inbound_nodes': []}, {'name': 'encoder', 'class_name': 'Model', 'config': {'name': 'encoder', 'layers': [{'name': 'input_4', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 256, 16, 1), 'dtype': 'float32', 'sparse': False, 'name': 'input_4'}, 'inbound_nodes': []}, {'name': 'conv2d_5', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_5', 'trainable': True, 'filters': 4, 'kernel_size': (3, 3), 'strides': (2, 2), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activit

In [20]:
# well i guess that's a no